#Parameters

In [99]:
IMGSIZE = 512
MNIST_SIZE = 28

MNIST_PATH = ''
MNIST_DATASET_PATH = 'dataset/mnist/'
TRAIN_DATASET_PATH = 'dataset/circle/' 


#Heat Diffusion Algorithm

In [100]:
import cv2 as cv
import numpy as np
from scipy.interpolate import RegularGridInterpolator
from scipy import ndimage
import imageio
import matplotlib.pyplot as plt
import matplotlib as mpl
import matplotlib.animation as animationt
mpl.rc('image', cmap='gray')
import math
import time

def imshow(img):
    plt.imshow(img)
    plt.colorbar()
    # plt.axis('off')
    plt.show()

def heat_kernel(size,delta):
    data = np.zeros((size, size))
    center = size // 2
    for i in range(size):
        for j in range(size):
            x = i - center
            y = j - center
            data[i, j] = np.exp(-(x**2 + y**2)/4)/(4*np.pi*delta**2)
    data /= np.sum(data)
    return data
    
def heat_kernel_convolution(image, kernel):
    """Compute heat kernel convolution on input image"""
    # Apply kernel convolution to image
    heat_convoluted = ndimage.convolve(image, kernel, mode='reflect')
    heat_convoluted = heat_convoluted.astype(float)

    # Set values above 0.5 to 1 and below to 0
    heat_convoluted[heat_convoluted > 0.3] = 1
    heat_convoluted[heat_convoluted <= 0.3] = 0

    return heat_convoluted

def heat_diffusion(image, kernel, lapse, filepath):
  """Apply heat diffusion on image over a period"""
  images = [image]
  for i in range(lapse):
    conv_img =  heat_kernel_convolution(images[i], kernel.data)
    #append the new convoluted image 
    images.append(conv_img)
  for img in images:
    img *= 255
  now = str(int(time.time()))
  filename = filepath+'_'+now+'.gif'
  imageio.mimsave(filename, images, duration = 100)
  return images


#Generate Heat Diffusion GIF Images
Method 1: based on random generated circle

In [101]:
# generate heat kernel
KERNEL_SIZE = 5
time_step = 5120
DELTA = IMGSIZE / time_step

KERNEL = heat_kernel(KERNEL_SIZE, DELTA)
print(KERNEL)

[[0.01247764 0.02641517 0.03391775 0.02641517 0.01247764]
 [0.02641517 0.05592091 0.07180387 0.05592091 0.02641517]
 [0.03391775 0.07180387 0.09219799 0.07180387 0.03391775]
 [0.02641517 0.05592091 0.07180387 0.05592091 0.02641517]
 [0.01247764 0.02641517 0.03391775 0.02641517 0.01247764]]


In [102]:
import os
import random

# generate random circles
def generate_circle_img():
    img = np.zeros((512,512))
    num_circles = random.randint(1,4)
    for n in range(num_circles):
      cc = (random.randint(100, 400),random.randint(100, 400))
      radius = random.randint(1, 100)
      cv.circle(img,cc,radius,(1,0,255),-1)
    # imshow(img)
    return img
    
def circle_gen():
    count = 30
    path = TRAIN_DATASET_PATH    
    if not os.path.exists(path):
        os.makedirs(path)
    for i in range(count):
      img = generate_circle_img()  
      heat_diffusion(img, KERNEL,100,path+str(KERNEL_SIZE)+'_'+str(DELTA))

In [103]:
circle_gen()

#Generate Heat Diffusion GIF Images
Method 2: based on MNIST dataset

In [104]:
import os
import torch
import struct
import numpy as np
from torch.utils.data import DataLoader,Dataset

#import MNIST dataset
class mnist_dataset(Dataset):
    def __init__(self,path,filekind='train'):
        self.data_path = path
        
        if filekind=='train' or filekind=='t10k':
            imgs_path = os.path.join(path,'%s-images-idx3-ubyte'%filekind)
            labels_path = os.path.join(path, '%s-labels-idx1-ubyte' % filekind)
        else:
            print("Error:filekind is only a string with 'train' or 't10k' ")
		
        with open(labels_path, 'rb') as lbpath:
            magic, n = struct.unpack('>II',lbpath.read(8))
            labels = np.fromfile(lbpath,dtype=np.uint8)

        with open(imgs_path, 'rb') as imgpath:
            magic, num, rows, cols = struct.unpack('>IIII',imgpath.read(16))
            images = np.fromfile(imgpath,dtype=np.uint8).reshape(len(labels), 784)

        self.labels = labels
        self.images = images

    def __getitem__(self, index):
        temp_label = self.labels[index]
        temp_image = self.images[index]
        temp_image = torch.tensor(temp_image.reshape(28,28),dtype=torch.float32)[None,...]

        return temp_label,temp_image

    def __len__(self):
        return len(self.labels)
        
def get_MNIST_data():
    my_mnist = mnist_dataset(MNIST_PATH,'train')
    return DataLoader(dataset=my_mnist, batch_size=20, shuffle=True)

def mnist_gen():
  m = get_MNIST_data()
  path = MNIST_DATASET_PATH   
  if not os.path.exists(path):
    os.makedirs(path)
  for label, image in m:
    image = image[0][0]/255
    heat_diffusion(image, KERNEL, 10, path+str(KERNEL_SIZE)+'_'+str(DELTA))

In [105]:
mnist_gen()

#Loading Data

In [106]:
import os
import torch
from torch.utils.data import DataLoader,Dataset
import torchvision.transforms as transforms
import cv2 as cv

# Define your dataset class
class mydataset(Dataset):
    def __init__(self, folder, transform=None):
        self.train_image_file_paths = [os.path.join(folder, image_file) for image_file in os.listdir(folder)]
        self.transform = transform
    
    def __len__(self):
        return len(self.train_image_file_paths)
    
    def __getitem__(self, idx):
        gif_root = self.train_image_file_paths[idx]
        gif_name = gif_root.split(os.path.sep)[-1]
        gif = cv.VideoCapture(gif_root)
        frames = []
        data = []
        while True:
            ret, frame = gif.read()
            if not ret:
                break
            frames.append(frame)
        gif.release()
       
        if self.transform is not None:
            while len(frames)<10:
                frames.append(frames[-1])
            for i in range(10):
                data.append(self.transform(frames[i]))
        if data is not None:
            data =  torch.stack(data)
        return data, KERNEL

# Set up transformations for your input images 
transform = transforms.Compose([
    transforms.ToPILImage(),
    transforms.ToTensor()
])

def get_train_data_loader():
    dataset = mydataset(TRAIN_DATASET_PATH, transform=transform)
    return DataLoader(dataset, batch_size=1, shuffle=True)

def get_MNIST_data_loader():
    dataset = mydataset(MNIST_DATASET_PATH, transform=transform)
    return DataLoader(dataset, batch_size=1, shuffle=True)



#CNN Model for MNIST

In [108]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Conv3d(10, 40, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(2,2)
        self.fc1 = nn.Linear(32 * 7 * 7, 64) 
        self.fc2 = nn.Linear(64,KERNEL_SIZE)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.where(x > 0.3, torch.ones_like(x), torch.zeros_like(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 7 * 7)  # Flatten the output for fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [109]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_loader = get_MNIST_data_loader()
    for inputs, kernel in train_loader:
        
        optimizer.zero_grad()
        outputs = model(inputs)
        kernel = kernel.to(torch.float32)
        loss = criterion(outputs, kernel)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), "kernel_prediction_model.pth")


Epoch 1/10, Loss: 0.06412512682378292
Epoch 2/10, Loss: 0.05962455913424492
Epoch 3/10, Loss: 0.05218198880553246
Epoch 4/10, Loss: 0.029241586811840536
Epoch 5/10, Loss: 0.009771335823461413
Epoch 6/10, Loss: 0.008208208908326924
Epoch 7/10, Loss: 0.004713819017633796
Epoch 8/10, Loss: 0.006245018257759511
Epoch 9/10, Loss: 0.004376534722978249
Epoch 10/10, Loss: 0.0016744563309475779


#CNN Model for Circle

In [110]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class CNN(nn.Module):
    def __init__(self):
        super(CNN, self).__init__()
        self.layer1 = nn.Conv3d(10, 40, kernel_size=3, padding=1)
        self.pool = nn.MaxPool3d(2,2)
        self.fc1 = nn.Linear(32 * 128 * 128, 64) 
        self.fc2 = nn.Linear(64,KERNEL_SIZE)

    def forward(self, x):
        x = self.layer1(x)
        x = torch.where(x > 0.5, torch.ones_like(x), torch.zeros_like(x))
        x = self.pool(x)
        x = x.view(-1, 32 * 128 * 128)  # Flatten the output for fully connected layers
        x = self.fc1(x)
        x = self.fc2(x)
        return x

In [111]:
import torch
import torch.nn as nn
from torch.autograd import Variable

# Initialize the model, loss function, and optimizer
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model = CNN().to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=0.001)

# Training loop
num_epochs = 10

for epoch in range(num_epochs):
    model.train()
    running_loss = 0.0
    train_loader = get_train_data_loader()
    for inputs, kernel in train_loader:
        
        optimizer.zero_grad()
        outputs = model(inputs)
        kernel = kernel.to(torch.float32)
        loss = criterion(outputs, kernel)
        loss.backward()
        optimizer.step()
        running_loss += loss.item()
    
    print(f"Epoch {epoch+1}/{num_epochs}, Loss: {running_loss/len(train_loader)}")

# Save the trained model
torch.save(model.state_dict(), "kernel_prediction_model.pth")


Epoch 1/10, Loss: 907.6780602662514
Epoch 2/10, Loss: 1110.113008034428
Epoch 3/10, Loss: 336.65137679974237
Epoch 4/10, Loss: 85.46268595637133
Epoch 5/10, Loss: 16.591602496132253
Epoch 6/10, Loss: 10.931503954287619
Epoch 7/10, Loss: 3.5950006987464924
Epoch 8/10, Loss: 2.324501742683351
Epoch 9/10, Loss: 1.4868542122095822
Epoch 10/10, Loss: 0.9220185413118451
